In [1]:
import torch
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_dense_adj
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

In [2]:
edge_index = torch.tensor([
    [0, 1],
    [0, 2],
    [0, 3],
    [1, 0],
    [2, 0],
    [3, 0]
], dtype=torch.long)

x = torch.tensor([[0, 1], [0, 1], [0, 1], [0, 1]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]

y = torch.tensor([[3.0], [1.0]]) # ranks

data = Data(x=x, edge_index=edge_index.t().contiguous(), y=y)

In [3]:
data.num_nodes, data.num_edges, data.num_node_features, data.has_isolated_nodes(), data.has_self_loops(), data.is_directed()

(4, 6, 2, False, False, False)

In [4]:
data.keys()

['y', 'x', 'edge_index']

In [5]:
adjacency = to_dense_adj(edge_index.t().contiguous())[0]

In [6]:
adjacency

tensor([[0., 1., 1., 1.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.],
        [1., 0., 0., 0.]])

In [7]:
data.y

tensor([[3.],
        [1.]])

In [15]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 16)
        self.conv2 = GCNConv(16, 16)
        self.out = torch.nn.Linear(16, data.num_node_features)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = gap(x, None)

        # print(x)
        # return F.log_softmax(x, dim=1)
        return self.out(x)

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
# data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    # loss = F.nll_loss(out, data.y)
    loss = F.mse_loss(out, data.y)
    print("Loss:", loss)
    loss.backward()
    optimizer.step()

/tmp/ipykernel_1209/3130078632.py:12: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = F.mse_loss(out, data.y)


Loss: tensor(4.8378, grad_fn=<MseLossBackward0>)
Loss: tensor(4.3658, grad_fn=<MseLossBackward0>)
Loss: tensor(4.8922, grad_fn=<MseLossBackward0>)
Loss: tensor(3.8940, grad_fn=<MseLossBackward0>)
Loss: tensor(3.9189, grad_fn=<MseLossBackward0>)
Loss: tensor(3.5488, grad_fn=<MseLossBackward0>)
Loss: tensor(3.4361, grad_fn=<MseLossBackward0>)
Loss: tensor(2.3193, grad_fn=<MseLossBackward0>)
Loss: tensor(2.3842, grad_fn=<MseLossBackward0>)
Loss: tensor(2.1622, grad_fn=<MseLossBackward0>)
Loss: tensor(2.6732, grad_fn=<MseLossBackward0>)
Loss: tensor(1.7890, grad_fn=<MseLossBackward0>)
Loss: tensor(2.2588, grad_fn=<MseLossBackward0>)
Loss: tensor(1.1236, grad_fn=<MseLossBackward0>)
Loss: tensor(1.5756, grad_fn=<MseLossBackward0>)
Loss: tensor(1.0346, grad_fn=<MseLossBackward0>)
Loss: tensor(1.0296, grad_fn=<MseLossBackward0>)
Loss: tensor(1.1503, grad_fn=<MseLossBackward0>)
Loss: tensor(1.0527, grad_fn=<MseLossBackward0>)
Loss: tensor(1.4495, grad_fn=<MseLossBackward0>)
Loss: tensor(1.4061,

In [24]:
edge_index = torch.tensor([
    [0, 1],
    [0, 2],
    [0, 3],
    [1, 0],
    [2, 0],
    [3, 0]
], dtype=torch.long)

x = torch.tensor([[0, 1], [0, 1], [0, 1], [0, 1]], dtype=torch.float) # configs: [0, 0, 0, 0], [1, 1, 1, 1]

test_data = Data(x=x, edge_index=edge_index.t().contiguous())

model(test_data)

tensor([[1.6490, 1.6121]], grad_fn=<AddmmBackward0>)